In [158]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import ast
import re
from torch.utils.data import TensorDataset, DataLoader

import logging
logging.basicConfig(level=logging.DEBUG)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [159]:
import pandas as pd
import torch
import re

def load_data(input_file, output_file, expected_landmarks=21):
    # Load input data
    input_df = pd.read_csv(input_file, header=None)
    
    # Convert each column of input data into a tensor and concatenate into a single tensor
    input_tensors = [torch.FloatTensor(input_df[col].values) for col in input_df.columns]
    inputs = torch.stack(input_tensors, dim=1)  # Stack columns into a single tensor
    
    # Load the output data as a DataFrame
    output_df = pd.read_csv(output_file, header=None)
    
    # Define a pattern to extract numbers from landmark strings
    pattern = r'Landmark_\d+:\s*\(([-\d.]+),\s*([-\d.]+),\s*([-\d.]+)\)'
    
    # Process each row in the output DataFrame
    all_landmarks = []
    for idx, row in output_df.iterrows():
        row_data = ','.join(map(str, row))  # Convert all elements to strings before joining
        matches = re.findall(pattern, row_data)
        if not matches:
            raise ValueError(f"No valid landmark data found in row {idx}: {row_data}")
        
        if len(matches) != expected_landmarks:
            raise ValueError(f"Expected {expected_landmarks} landmarks, but found {len(matches)} in row {idx}")
        
        landmarks = [float(coord) for match in matches for coord in match]
        all_landmarks.append(landmarks)
    
    # Convert list of landmarks into a tensor
    targets = torch.FloatTensor(all_landmarks)
    
    if targets.shape[1] != expected_landmarks * 3:
        raise ValueError(f"Inconsistent number of coordinates. Expected {expected_landmarks * 3}, but got {targets.shape[1]}")
    
    return inputs, targets

# Usage example


In [160]:
class LandmarkPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LandmarkPredictor, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        self.bn = nn.BatchNorm1d(hidden_size)
        self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(hidden_size, output_size)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.bn(self.hidden(x)))
        x = self.dropout(x)
        x = self.output(x)
        return x

In [161]:
def train_model(model, train_loader, epochs=1000, lr=0.01):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        running_loss = 0.0
        running_mae = 0.0
        
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_mae += torch.mean(torch.abs(outputs - targets)).item()

        epoch_loss = running_loss / len(train_loader)
        epoch_mae = running_mae / len(train_loader)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy (MAE): {epoch_mae:.4f}')

In [164]:
if __name__ == "__main__":
    try:
        inputs, targets = load_data("hand_landmarks.csv", "frame_landmarks.csv")
        print(f"Inputs shape: {inputs.shape}")
        print(f"Targets shape: {targets.shape}")
    except ValueError as e:
        print(f"Error processing data: {e}")
    input_size = inputs.shape[1]  
    hidden_size = 64
    output_size = targets.shape[1] 
    print(output_size)

Error processing data: Expected 21 landmarks, but found 20 in row 1650
63


In [163]:
if __name__ == "__main__":
    try:
        inputs, targets = load_data("hand_landmarks.csv", "frame_landmarks.csv")
        print(f"Inputs shape: {inputs.shape}")
        print(f"Targets shape: {targets.shape}")
    except ValueError as e:
        print(f"Error processing data: {e}")
    input_size = inputs.shape[1]  
    hidden_size = 64
    output_size = targets.shape[1]  # Change this to use the second dimension

    # Create dataset and dataloader
    dataset = TensorDataset(inputs, targets)
    train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

    model = LandmarkPredictor(input_size, hidden_size, output_size).to(device)

    train_model(model, train_loader)

    # Save model
    torch.save(model.state_dict(), "./models/landmark_predictor.pth")
    print("Model trained and saved as 'landmark_predictor.pth'")

Error processing data: Expected 21 landmarks, but found 20 in row 1650
Epoch [10/1000], Loss: 0.0049, Accuracy (MAE): 0.0460
Epoch [20/1000], Loss: 0.0043, Accuracy (MAE): 0.0427
Epoch [30/1000], Loss: 0.0042, Accuracy (MAE): 0.0426
Epoch [40/1000], Loss: 0.0041, Accuracy (MAE): 0.0418


KeyboardInterrupt: 